## Project Thesis ~ Initial Structure

## Problem Types

 - Classification 
 - Regression
 - Clustering
 - Dimension Reduction
 - Data Visualization & Analysis
 

In [2]:
# Libraries

# Core Libraries
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Helpers
import pprint
from dateutil import parser
import ijson
import json
from datetime import datetime

# Visions | Data Type Detection
from visions.functional import detect_type, infer_type, cast_to_inferred
from visions.typesets import StandardSet
from visions.typesets import CompleteSet

# YData Profiling
from ydata_profiling import ProfileReport


## Define the Problem - Clear Objectives/Requirements
Problems before requirements, requirements before solutions, solutions before design, and design before technology.

- Problem types | Classification, Regression, Clustering, Dimension Reduction, Data Visualizations & Techniques
- Target
- Models | Random Forest | Optional

In [3]:
# Problem | {Problem Type, Target Variable}
def df_objective (problem_type, target_variable):

    return {
        'problem_type' : problem_type,
        'target_feature' : target_variable
    }

### Objective | Sample Data

In [4]:
# Titanic
tn_objective = df_objective('classification', 'Survived')

## Data Gathering - Tabular Data

**File Data Types**: json, csv, xlsx, xml, dataframe,

**API**: url link, http, ftp

**Category**: Web page, Image, Audio, Video

In [5]:
# File Characteristics
def file_descr (file_path): 
    descr = {
        'FileType': os.path.splitext(file_path)[1],
        'FileSize': os.path.getsize(file_path),
    }
    return descr

In [6]:
# Converters | File type to Dataframe

# Covert File to Dataframe
def df_convert (file_path, file_descr):

    # JSON
    if (file_descr['FileType'] == '.json'):

        # Open File and Convert it to JSON Object
        with open(file_path, 'r') as file:
            data = ijson.items(file, 'item')
            json_object = []
            count = 0
            print(data)
            for line in data:
                json_object.append(line)
                count = count + 1
                if(count == 1000): break
        return pd.DataFrame(json_object)

    # CSV
    elif (file_descr['FileType'] == '.csv'):

        # Check whether there is a header | TODO
        return pd.read_csv(file_path)
    else: 
        return False

In [7]:
## Gather Data
def df_gather(fp):

    # File Characteristics
    file_characteristics = file_descr(fp)

    # Dataframe Initialization 
    df = df_convert(fp, file_characteristics)

    return df

### Data Gathering | Sample Data

In [7]:
# Wasabi
df_ws = df_gather('data/sample_data/Wasabi/batterytesters_dataset.json')

In [ ]:
# House Prices
df_hp = df_gather('data/sample_data/house_price_data/train.csv')

In [ ]:
# Credit Card Fraud
df_cc = df_gather('data/sample_data/credit_card_fraud/creditcard.csv')

In [ ]:
# Modcloth | TOCHECK
file_path_md = 'data/sample_data/modcloth/modcloth_final_data.json'
df_md = pd.read_json(file_path_md, lines=True)

In [8]:
# Titanic 
df_tn = df_gather('data/sample_data/titanic/train.csv')

## Data Profiling

### Dataset
- Shape

In [9]:
# Features, Rows
def df_shape(df):
    return {
        'features': df.shape[1],
        'rows': df.shape[0]
    }

### Features
- Role | ID, Input (Indpendant), Target (Dependant)
- Data Type | Integer, Float, Boolean, Categorical, Complex, DateTime, Object, String
- Feature Type | Categorical (Ordinal, Binary), Numerical
- Level | Nominal, Interval, Ordinal, Binary

In [10]:
# Data Type | Standard Set | Integer,  Float, Boolean, Categorical, Complex, DateTime, Object, String
typeset = StandardSet()
def f_data_type(f):
    return str((detect_type(f, typeset)))

In [11]:
# Feature Type | Categorical, Numerical, Alphanumerical | {Data Type, Unique values, Thres}
def f_feature_type(data_type, unique_values, threshold=10): 

    if ((data_type == 'String' or 
         data_type == 'Integer' or 
         data_type == 'Float') and unique_values < threshold):
        return 'Categorical'
    elif (data_type == 'Int' or data_type == 'Float'):
        return 'Numerical'
    else: 
        return 'Alphanumerical'

In [12]:
# Qualitative Characteristics | Nominal, Interval, Ordinal, Binary
def f_qual(data_type):
    if (data_type == 'String'):
        return 'Nominal'
    elif (data_type == 'Int' or data_type == 'Float'):
        return 

In [14]:
# Role | Input (Independant), ID (Unique Identifier)
def f_role(f):
    
    f_unique_values = len(f.unique())
    f_total_values = len(f)

    if (f_unique_values / f_total_values) > 0.9:
        return 'id'
    else: 
        return 'input' 

In [15]:
## Feature Profiling 
def f_profile(f):
    
    # Data Type
    data_type = f_data_type(f)

    # Role
    role = f_role(f)

    # Unique Values
    unique_values = len(f.value_counts())

    # Feature Type
    feature_type = f_feature_type(data_type, unique_values, 10)

    return {
        'data_type' : data_type,
        'role' : role,
        'unique_values' : unique_values,
        'feature_type' : feature_type
    }

In [16]:
## All feature profiling
def df_profile(df, f_target):
    df_profile = {}

    # Dataset
    df_profile['dataset'] = df_shape(df)

    # Target Feature
    df_profile['target_feature'] = f_target

    # Features
    df_features = {}
    for f in df.columns: 
        df_features[f] = f_profile(df[f])
    df_profile['features'] = df_features

    return df_profile 

### Feature Profiling | Samples Data

In [ ]:
# House Prices
hp_profiling = df_profile(df_hp)

In [ ]:
# Modcloth
md_profiling = df_profile(df_md)

In [17]:
# Titanic 
tn_profile = df_profile(df_tn, tn_objective['target_feature'])

## Exploratory Analysis & Visualization | Descriptive Statistics

### Dataset
- Duplicates


In [18]:
## Duplicates | Check if they exist
def df_duplicates(df):
    return {
        'exist' : df.duplicated().any(),
        'sum' : df.duplicated().sum()
    } 

### Univariate Analysis 

- Interval, Ordinal Statistics | Count, Mean,  Std, Min, 25%, 50%, 75%, Max
- Missing Values 
- Outliers
- Histogram
- Box Plot

In [19]:
# Statistics
def f_statistics(f, d_type, f_type):

    f_statistics = {}
    if f_type == 'Numerical' and d_type != 'String':

        descr = f.describe()
        f_statistics = {
            'count' : int(descr['count']),
            'mean' : round(descr['mean'], 2),
            'std' : round(descr['std'], 2),
            'min' : round(descr['min'], 2),
            'max' : round(descr['max'], 2)
        }
    elif f_type == 'Categorical': 
        for i, y in f.value_counts().items():
            
            # Calculate Frequency of each categorical value
            freq = round(y / len(f), 2)
            f_statistics[i] = {
                'value': y,
                'frequency': freq
            }

    return f_statistics

In [20]:
# Missing Values | Return: {Total Missing Values, Percentage}
def f_missing_data(f):

    ## Missing Values
    missing_values = f.isnull().sum()

    ## Percentage
    percentage = round((missing_values / len(f)), 2)
    return {
        'missing_values': missing_values,
        'percentage': percentage
    }

In [22]:
## Univariate | Feature | {Dataframe_Feature, Data Type, Feature Type}
def f_univariate(f, d_type, f_type): 
    f_univariate = {}

    # Statistics
    f_univariate['statistics'] = f_statistics(f, d_type, f_type)

    # Missing Values
    f_univariate['missing_data'] = f_missing_data(f)

    return f_univariate

In [23]:
# Univariate | Dataframe | {Dataframe, Dataframe_Profiling}
def df_univariate(df, df_prof):

    df_univariate = {}
    for f, d in df_prof['features'].items():
        df_univariate[f] = f_univariate(df[f], d['data_type'], d['feature_type'])

    df_univariate['features'] = df_univariate

    return df_univariate

### Bivariate Analysis
 - How each variable correlates to target variable

In [24]:
# Correlation Analysis | Each feature with target variable
def f_corr (f, corr_matrix):
    if f in corr_matrix:
        return round(corr_matrix[f], 2)
    else:
        return ''

In [25]:
# Bivariate Analysis | Dataframe
def df_bivariate(df, df_prof):

    df_bivariate = {}

    # Correlation Analysis
    corr_matrix = df.corr(numeric_only=True)[df_prof['target_feature']]
    for f in df_prof['features']:
        df_bivariate[f] = f_corr(f, corr_matrix)
    
    return df_bivariate

### Multivariate Analysis
- Normality -> Data should look like normal distribution
- Homoscedasticity -> 
- Linearity -> Linear Patterns
- 

## EDA 

In [26]:
# EDA  | {Datframe, Dataframe Profiling}
def df_eda (df, df_prof):

    df_eda = {}

    # Duplicates
    df_eda['duplicates'] = df_duplicates(df)

    # Univariate
    df_eda['univariate'] = df_univariate(df, df_prof)

    # Bivariate
    df_eda['bivariate'] = df_bivariate(df, df_prof)
 
    return  df_eda

In [41]:
# Titanic
tn_eda = df_eda(df_tn, tn_profile)

## Data Description
Dataset description in one dictionary

In [34]:
# Dataset Description
def df_descr(df, df_objective):

    # Profile 
    df_prof = df_profile(df, df_objective['target_feature'])

    # Statistics
    df_stat = df_eda(df, df_prof)

    # Duplicates
    df_prof['dataset']['duplicates'] = df_stat['duplicates']

    # Univariate
    for f, i in df_prof['features'].items():
        df_prof['features'][f]['eda'] = df_stat['univariate'][f]

    return df_prof

### Descriptive Statistics | Sample Data

In [30]:
# Titanic
tn_descr = df_descr(df_tn, tn_objective)

## Data Cleaning & Correcting & Formatting & Completing (Transform)

### Drop Duplicates

In [37]:
# Drop Duplicates
def drop_duplicates(df): 
    try: 
        df.drop_duplicates(inplace=True)
        return True
    except ValueError:
        return ValueError

### Data Imputation ~ Handle Missing Values

Methods: Mean, Median, KNN, Most Frequent Value, Random Numbers between mean & std, Exploit correlated feature(s)

In [45]:
# Handle Missing Values | {Dataframe, Profile, EDA}

def df_handle_missing_values(df: pd.DataFrame, df_profile: dict, df_eda: dict):

    # Split Categorical and Numerical Columns
    numerical_columns = []
    categorical_columns = []

    # For each feature
    for f, p in df_profile['features'].items():

        # If missing values are above 90% | Drop
        if (df_eda['univariate'][f]['missing_data']['percentage'] >= 0.9):
            df.drop(columns=[f], inplace=True)

        # Split | If missing percentage is above 0
        if (df_eda['univariate'][f]['missing_data']['missing_values'] > 0.0):
            if (p['feature_type'] == 'Numerical'):
                numerical_columns.append(f)
            elif (p['feature_type'] == 'Categorical'):
                categorical_columns.append(f)

    # Imputers
    numerical_imputer = SimpleImputer(strategy='mean')
    categorical_imputer = SimpleImputer(strategy='most_frequent')

    # Mean | Numerical
    if (len(numerical_columns) > 0):
        df[numerical_columns] = numerical_imputer.fit_transform(df[numerical_columns])

    # Mode | Categorical
    if (len(categorical_columns) > 0):
        df[categorical_columns] = categorical_imputer.fit_transform(df[categorical_columns])

    return df

### Sample Data | Data Imputation

In [46]:
# Titanic
tn_imputed = df_handle_missing_values(df_tn, tn_profile, tn_eda)
tn_imputed_descr = df_descr(tn_imputed, tn_objective)

### Data Anomaly Detection ~ Handle Outliers

- Interquartile Range (IQR) method 

In [49]:
# Handle Outliers | {Dataframe, Profile, EDA, Threshold}
def df_handle_outliers(df : pd.DataFrame, df_profile : dict, df_eda : dict, threshold : float = 1.5):

    # Numerical Columns
    numerical_columns = []

    for f, p in df_profile['features'].items():
        if (p['feature_type'] == 'Numerical'):
            numerical_columns.append(f)

    for f in numerical_columns: 
        Q1 = df[f].quantile(0.25)
        Q3 = df[f].quantile(0.75)

        IQR = Q3 - Q1 
        outliers = ((df[f] < (Q1 - threshold * IQR)) | (df[f] > (Q3 + threshold * IQR)))
        df = df[~outliers]
    
    return df

### Sample Data | Data Anomaly Detection

In [55]:
# Titanic
tn_anomaly = df_handle_outliers(df_tn, tn_profile, tn_eda, 3)
tn_anomaly_descr = df_descr(tn_anomaly, tn_objective)

## Feature Engineering

- Feature Extraction
- Feature Selection | Either in data-preprocessing, or the classifier

In [ ]:
## Feature Selection | Remove features with low or zero variance